# Documentation introspection Aalysis

1. Get the documentation URL:
    This can be done by using the documentation path from the lib. I the case of pytorch, it is
    `https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear`
    and for pygnn it is `https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv`
    In both cases we just need to subtitute the URL hash fragment in the end
2. Get the docs strign andn remove unwanted sections from it, such as the Examples section
3. Figure out how many inputs and outputs the graph editor node component will have 

## 1. Get the documentation URL:

In [4]:
from app.features.model.generate import layers, featurizers

def is_from_pygnn(class_path: str) -> bool:
    return class_path.startswith('torch_geometric.')

def is_from_pytorch(class_path: str) -> bool:
    return class_path.startswith('torch.')

def get_documentation_link(component) -> str:
    if is_from_pygnn(component.name):
        return f'https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#{component.name}'
    elif is_from_pytorch(component.name):
        return f'https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#{component.name}'
    return None
    
print([ get_documentation_link(comp) for comp in layers + featurizers ])

[None, None, 'https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear', 'https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Sigmoid', 'https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.ReLU', 'https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.GCNConv', None]


In [6]:
import pandas as pd

from app.features.model.generate import layers, featurizers
from app.features.model.utils import get_class_from_path_string


alldocs = [
    (layer.name, get_class_from_path_string(layer.name).__doc__)
    for layer in layers + featurizers
]
def get_by_first_start(tuples, first_start):
    return [
        item
        for item in tuples
        if item[0].startswith(first_start)
    ]

torch_geometric_comps = get_by_first_start(alldocs, 'torch_geometric.')
torch_comps = get_by_first_start(alldocs, 'torch.')
app_comps = get_by_first_start(alldocs, 'app.')

def has_examples(docs):
    return 'Examples:' in docs


def has_args(docs):
    return 'Args:\n' in docs

def has_docs(docs):
    return docs is not None and len(docs) > 0

df = pd.DataFrame({ 'src_lib': [], 'docs': [], 'has_examples': [], 'has_args': [], 'has_examples': [] })
def info(title, stuff, df):
    for (class_path, docs) in stuff:
        df.loc[class_path] = {
            'docs': docs,
            'src_lib': title,
            'has_args': int(has_args(docs)),
            'has_examples': int(has_examples(docs))
        }
    
info('pygnn', torch_geometric_comps, df)
info('torch', torch_comps, df)
info('app', app_comps, df)
df.loc[:]

,src_lib,docs,has_examples,has_args
torch_geometric.nn.GCNConv,pygnn,"The graph convolutional operator from the `""Se...",0,1
torch.nn.Linear,torch,Applies a linear transformation to the incomin...,1,1
torch.nn.Sigmoid,torch,Applies the element-wise function:\n\n .. m...,1,0
torch.nn.ReLU,torch,Applies the rectified linear unit function ele...,1,1
app.features.model.layers.GlobalPooling,app,A global pooling module that wraps the usage o...,0,1
app.features.model.layers.Concat,app,\n A helper layer that concatenates the out...,0,0
app.features.model.featurizers.MoleculeFeaturizer,app,\n Small molecule featurizer.\n Args:\n ...,0,1


In [2]:
print(df.loc['torch.nn.Linear', 'docs'])

Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

    This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(N, *, H_{in})` where :math:`*` means any number of
          additional dimensions and :math:`H_{in} = \text{in\_features}`
        - Output: :math:`(N, *, H_{out})` where all but the last dimension
          are the same shape as the input and :math:`H_{out} = \text{out\_features}`.

    Attributes:
        weight: the learnable weights of the module of shape
            :math:`(\text{out\_features}, \text{in\_features})`. The values are
            initialized from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})`, where
            :math:`k = \frac{1}{\text{in\_features}}`
        bias:   the learnable bias 

In [3]:
print(df.loc['torch_geometric.nn.GCNConv', 'docs'])

The graph convolutional operator from the `"Semi-supervised
    Classification with Graph Convolutional Networks"
    <https://arxiv.org/abs/1609.02907>`_ paper

    .. math::
        \mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},

    where :math:`\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}` denotes the
    adjacency matrix with inserted self-loops and
    :math:`\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}` its diagonal degree matrix.
    The adjacency matrix can include other values than :obj:`1` representing
    edge weights via the optional :obj:`edge_weight` tensor.

    Its node-wise formulation is given by:

    .. math::
        \mathbf{x}^{\prime}_i = \mathbf{\Theta}^{\top} \sum_{j \in
        \mathcal{N}(v) \cup \{ i \}} \frac{e_{j,i}}{\sqrt{\hat{d}_j
        \hat{d}_i}} \mathbf{x}_j

    with :math:`\hat{d}_i = 1 + \sum_{j \in \mathcal{N}(i)} e_{j,i}`, where
    :math:`e_{j,i}` denotes the edge weight from

### Removing Sections of the docs
To make the doc. simpler, we need to remove sections that refer exclusivelly to coding such as the Examples section

As can be seen, documentation is divided by indentation blocks.

So an ideia to remove sections is to find it's title, and the next indentation block. then slice it off

In [4]:

def count_tabs(line: str):
    # naive loop over chars
    total = 0
    for c in line:
        if c == ' ' or c == '\t':
            total += 1
        else:
            break
    return total

def remove_indentation_of_section(text: str, section_title: str) -> str:
    lines = text.split('\n')
    start_idx = None
    tab_size = None
    end_idx = None
    for idx, line in enumerate(lines):
        if section_title in line:
            start_idx = idx
            end_idx = start_idx
            tab_size = count_tabs(line) + 4
            continue
        if start_idx is not None and count_tabs(line) >= tab_size:
            end_idx += 1
        elif start_idx is not None:
            break
    if start_idx is None:
        return text
    return '\n'.join(lines[:start_idx] + lines[end_idx+1:])
def test_remove_indentation():
    assert remove_indentation_of_section('''The graph convolutional operator from the `"Semi-supervised
    Classification with Graph Convolutional Networks"
    <https://arxiv.org/abs/1609.02907>`_ paper

    .. math::
        \mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},

    where :math:`\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}` denotes the
    adjacency matrix with inserted self-loops and
    :math:`\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}` its diagonal degree matrix.
    The adjacency matrix can include other values than :obj:`1` representing
    edge weights via the optional :obj:`edge_weight` tensor.

    Its node-wise formulation is given by:

    .. math::
        \mathbf{x}^{\prime}_i = \mathbf{\Theta}^{\top} \sum_{j \in
        \mathcal{N}(v) \cup \{ i \}} \frac{e_{j,i}}{\sqrt{\hat{d}_j
        \hat{d}_i}} \mathbf{x}_j

    with :math:`\hat{d}_i = 1 + \sum_{j \in \mathcal{N}(i)} e_{j,i}`, where
    :math:`e_{j,i}` denotes the edge weight from source node :obj:`j` to target
    node :obj:`i` (default: :obj:`1.0`)

    Args:
        in_channels (int): Size of each input sample, or :obj:`-1` to derive
            the size from the first input(s) to the forward method.
        out_channels (int): Size of each output sample.
        improved (bool, optional): If set to :obj:`True`, the layer computes
            :math:`\mathbf{\hat{A}}` as :math:`\mathbf{A} + 2\mathbf{I}`.
            (default: :obj:`False`)
        cached (bool, optional): If set to :obj:`True`, the layer will cache
            the computation of :math:`\mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
            \mathbf{\hat{D}}^{-1/2}` on first execution, and will use the
            cached version for further executions.
            This parameter should only be set to :obj:`True` in transductive
            learning scenarios. (default: :obj:`False`)
        add_self_loops (bool, optional): If set to :obj:`False`, will not add
            self-loops to the input graph. (default: :obj:`True`)
        normalize (bool, optional): Whether to add self-loops and compute
            symmetric normalization coefficients on the fly.
            (default: :obj:`True`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.

    Shapes:''', 'Args') == '''The graph convolutional operator from the `"Semi-supervised
    Classification with Graph Convolutional Networks"
    <https://arxiv.org/abs/1609.02907>`_ paper

    .. math::
        \mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},

    where :math:`\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}` denotes the
    adjacency matrix with inserted self-loops and
    :math:`\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}` its diagonal degree matrix.
    The adjacency matrix can include other values than :obj:`1` representing
    edge weights via the optional :obj:`edge_weight` tensor.

    Its node-wise formulation is given by:

    .. math::
        \mathbf{x}^{\prime}_i = \mathbf{\Theta}^{\top} \sum_{j \in
        \mathcal{N}(v) \cup \{ i \}} \frac{e_{j,i}}{\sqrt{\hat{d}_j
        \hat{d}_i}} \mathbf{x}_j

    with :math:`\hat{d}_i = 1 + \sum_{j \in \mathcal{N}(i)} e_{j,i}`, where
    :math:`e_{j,i}` denotes the edge weight from source node :obj:`j` to target
    node :obj:`i` (default: :obj:`1.0`)


    Shapes:'''
    
test_remove_indentation()

In [ ]:
# TODO: make sure Examples sections are removed